In [0]:
import h5py as h5
import numpy as np
from google.colab import drive

**Convert H5 file to Test & Train data**

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with h5.File('/content/drive/My Drive/SVHN_single_grey1.h5') as hdf:
  ls=list(hdf.keys())
  print(ls)
  X_data_test=hdf.get('X_test')
  X_data_train=hdf.get('X_train')
  Y_data_test=hdf.get('y_test')
  Y_data_train=hdf.get('y_train')
  val_x=hdf.get('X_val')
  val_y=hdf.get('y_val')
  X_test=np.array(X_data_test)
  Y_test=np.array(Y_data_test)
  X_train=np.array(X_data_train)
  Y_train=np.array(Y_data_train)
  X_val=np.array(val_x)
  y_val=np.array(val_y)
  print(X_test.shape)

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']
(18000, 32, 32)


In [0]:
X_train.shape

(42000, 32, 32)

In [0]:
X_test.shape


(18000, 32, 32)

In [0]:
Y_train.shape

(42000,)

**Convert 3D to 2D array - As Machine learning algorith such as KNN supports 2D array**

In [0]:
nsamples, nx, ny = X_train.shape
X_train_2d = X_train.reshape((nsamples,nx*ny))

In [0]:
nsamples, nx, ny = X_test.shape
X_test_2d=X_test.reshape((nsamples,nx*ny))

In [0]:
X_train_2d.shape

(42000, 1024)

**Scaling data**

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train_2d)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(X_train_2d)
test_img = scaler.transform(X_test_2d)

**Apply PCA to reduce features**

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca=PCA(0.90)

In [0]:
pca.fit(X_train_2d)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [0]:
train_img = pca.transform(X_train_2d)
test_img = pca.transform(X_test_2d)

**Image Classification using KNN**

**Finding Best hyper parameter using GridSearch CV**

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

grid_params={'n_neighbors':[3,5,11,19],
            'weights':['uniform','distance'],
            'metric':['euclidean','manhattan']}
gs=GridSearchCV(KNeighborsClassifier(),grid_params,verbose=1,cv=3,n_jobs=-1)
gs_results=gs.fit(train_img,Y_train)

gs_results.best_score_
gs_results.best_estimator_
gs_results.best_params_

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  9.8min finished


{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}

**Applying hyper parameters in KNN model**

In [0]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(metric= 'manhattan', n_neighbors= 19, weights= 'distance')

#Train the model using the training sets
knn.fit(train_img, Y_train)

#Predict the response for test dataset
y_pred = knn.predict(test_img)

In [0]:
y_pred

array([9, 2, 2, ..., 7, 9, 3], dtype=uint8)

In [0]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))

Accuracy: 0.5674444444444444
